# Coursera Data Science Capstone

#### Notebook used for the Final Project of the IBM Data Science Professional Certificate

#### Import the required libraries

In [49]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import requests
!pip install geocoder
import geocoder
!pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

#### Download the data and format it

In [50]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json

In [51]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [52]:
neighborhoods_data = newyork_data['features']

#### Create a dataframe and a function to fill the data into the dataframe

In [53]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [54]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [55]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [56]:
NY_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)

#### Using geopy library to get the latitude and longitude values of New York City

In [57]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Creating a map of Brooklyn with neighborhoods

In [58]:
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(NY_data['Latitude'], NY_data['Longitude'], NY_data['Borough'], NY_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

### Using the Foursquare API

#### We are going to start using the Foursquare API to explore the neighborhoods and segment them
First we define our credentials (kept secret here)

In [59]:
# @hidden-cell
CLIENT_ID = 'WDMQ2G40J0ZUTBBIVTMJ5WHFEUYDSR3FSLQML0WK2CSBXOQB' # Foursquare ID
CLIENT_SECRET = 'UBR55TPVIDRQ4RDO0IB320B3M0512K0NG1MUSJ3R2HBVRPAE' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 150 # A default Foursquare API limit value

#### Function that extracts the category of the venue

In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=350):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Create a new dataframe called  _NY_venues_ and fill it with the dataset information

In [61]:
NY_venues = getNearbyVenues(names=NY_data['Neighborhood'],
                                   latitudes=NY_data['Latitude'],
                                   longitudes=NY_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


#### Size of the resulting dataframe and how many unique categories there are

In [62]:
print(NY_venues.shape)
NY_venues.head()

(1620, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
2,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
3,Bay Ridge,40.625801,-74.030621,Georgian Dream Cafe and Bakery,40.625586,-74.030196,Caucasian Restaurant
4,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar


In [63]:
print('There are {} uniques categories.'.format(len(NY_venues['Venue Category'].unique())))

There are 249 uniques categories.


### Analyzing neighborhoods

In [64]:
# one hot encoding
NY_onehot = pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NY_onehot['Neighborhood'] = NY_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [NY_onehot.columns[-1]] + list(NY_onehot.columns[:-1])
NY_onehot = NY_onehot[fixed_columns]

NY_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Indoor Play Area,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### New dataframe size

In [65]:
NY_onehot.shape

(1620, 249)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [66]:
NY_grouped = NY_onehot.groupby('Neighborhood').mean().reset_index()
# NY_grouped

In [67]:
NY_grouped.shape

(70, 249)

#### Print each neighborhood along with the top 5 most common venues

In [73]:
num_top_venues = 5

for hood in NY_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = NY_grouped[NY_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                venue  freq
0         Gas Station  0.11
1  Italian Restaurant  0.11
2            Pharmacy  0.11
3         Pizza Place  0.06
4          Bagel Shop  0.06


----Bay Ridge----
                venue  freq
0  Italian Restaurant  0.12
1                 Bar  0.07
2         Pizza Place  0.05
3    Greek Restaurant  0.05
4      Ice Cream Shop  0.05


----Bedford Stuyvesant----
                venue  freq
0         Coffee Shop   0.2
1          Playground   0.1
2  Italian Restaurant   0.1
3         Pizza Place   0.1
4       Deli / Bodega   0.1


----Bensonhurst----
                 venue  freq
0   Italian Restaurant  0.33
1             Pharmacy  0.17
2  American Restaurant  0.17
3         Liquor Store  0.17
4            Pet Store  0.17


----Bergen Beach----
               venue  freq
0     Baseball Field   1.0
1        Yoga Studio   0.0
2  Mobile Phone Shop   0.0
3      Movie Theater   0.0
4      Moving Target   0.0


----Boerum Hill----
                    venue

#### Put everything into a new dataframe

In [74]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [75]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = NY_grouped['Neighborhood']

for ind in np.arange(NY_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pharmacy,Gas Station,Italian Restaurant,Pizza Place,Restaurant,Kebab Restaurant,Russian Restaurant,Bagel Shop,Liquor Store,Donut Shop
1,Bay Ridge,Italian Restaurant,Bar,Grocery Store,Spa,Pizza Place,Ice Cream Shop,Greek Restaurant,Hookah Bar,Spanish Restaurant,Bookstore
2,Bedford Stuyvesant,Coffee Shop,Playground,Boutique,Bar,Juice Bar,Café,Italian Restaurant,Deli / Bodega,Pizza Place,Field
3,Bensonhurst,Italian Restaurant,American Restaurant,Pharmacy,Pet Store,Liquor Store,Women's Store,Factory,Falafel Restaurant,Farm,Farmers Market
4,Bergen Beach,Baseball Field,Women's Store,Event Service,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant


#### Sorting the places where the competition is not one of the 10 most common places in the area

In [76]:
sorted_neigh = neighborhoods_venues_sorted.loc[~neighborhoods_venues_sorted['1st Most Common Venue'].isin(['Bakery','Donut Shop', 'Bagel Shop', 'Coffee Shop', 'Café', 'Beach', 'Chinese Restaurant'])]
sorted_neigh = sorted_neigh.loc[~sorted_neigh['2nd Most Common Venue'].isin(['Bakery','Donut Shop', 'Bagel Shop', 'Coffee Shop', 'Café',  'Beach'])]
sorted_neigh = sorted_neigh.loc[~sorted_neigh['3rd Most Common Venue'].isin(['Bakery','Donut Shop', 'Bagel Shop', 'Coffee Shop', 'Café',  'Beach'])]
sorted_neigh = sorted_neigh.loc[~sorted_neigh['4th Most Common Venue'].isin(['Bakery','Donut Shop', 'Bagel Shop', 'Coffee Shop', 'Café',  'Beach',  'Beach'])]
sorted_neigh = sorted_neigh.loc[~sorted_neigh['5th Most Common Venue'].isin(['Bakery','Donut Shop', 'Bagel Shop', 'Coffee Shop', 'Café',  'Beach'])]
sorted_neigh = sorted_neigh.loc[~sorted_neigh['6th Most Common Venue'].isin(['Bakery','Donut Shop', 'Bagel Shop', 'Coffee Shop', 'Café',  'Beach'])]
sorted_neigh = sorted_neigh.loc[~sorted_neigh['7th Most Common Venue'].isin(['Bakery','Donut Shop', 'Bagel Shop', 'Coffee Shop', 'Café',  'Beach'])]
sorted_neigh = sorted_neigh.loc[~sorted_neigh['8th Most Common Venue'].isin(['Bakery','Donut Shop', 'Bagel Shop', 'Coffee Shop', 'Café',  'Beach'])]
sorted_neigh = sorted_neigh.loc[~sorted_neigh['9th Most Common Venue'].isin(['Bakery','Donut Shop', 'Bagel Shop', 'Coffee Shop', 'Café',  'Beach'])]
sorted_neigh = sorted_neigh.loc[~sorted_neigh['10th Most Common Venue'].isin(['Bakery','Donut Shop', 'Bagel Shop', 'Coffee Shop', 'Café',  'Beach'])].reset_index(drop=True)

sorted_neigh

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Italian Restaurant,Bar,Grocery Store,Spa,Pizza Place,Ice Cream Shop,Greek Restaurant,Hookah Bar,Spanish Restaurant,Bookstore
1,Bensonhurst,Italian Restaurant,American Restaurant,Pharmacy,Pet Store,Liquor Store,Women's Store,Factory,Falafel Restaurant,Farm,Farmers Market
2,Bergen Beach,Baseball Field,Women's Store,Event Service,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant
3,Borough Park,Bank,Deli / Bodega,Fast Food Restaurant,Bistro,Electronics Store,Restaurant,Pizza Place,Pharmacy,Event Service,Fish Market
4,Clinton Hill,Mexican Restaurant,Thai Restaurant,Restaurant,Italian Restaurant,Pizza Place,Yoga Studio,Indian Restaurant,Cajun / Creole Restaurant,Pie Shop,Caribbean Restaurant
5,East New York,Deli / Bodega,Gym,Convenience Store,Chinese Restaurant,Event Service,Business Service,Food Truck,Spanish Restaurant,Gym / Fitness Center,Grocery Store
6,Erasmus,Caribbean Restaurant,Pizza Place,Juice Bar,Convenience Store,Chinese Restaurant,Sandwich Place,Food Truck,Fried Chicken Joint,Liquor Store,Dentist's Office
7,Flatlands,Fried Chicken Joint,Dessert Shop,Fast Food Restaurant,Dry Cleaner,Paper / Office Supplies Store,Caribbean Restaurant,Park,Asian Restaurant,Pharmacy,Deli / Bodega
8,Gerritsen Beach,Harbor / Marina,Gas Station,Women's Store,Ethiopian Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant
9,Highland Park,Spanish Restaurant,Park,Tennis Court,Gym / Fitness Center,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


### List of possible locations

In [77]:
sorted_neigh['Neighborhood']

0             Bay Ridge
1           Bensonhurst
2          Bergen Beach
3          Borough Park
4          Clinton Hill
5         East New York
6               Erasmus
7             Flatlands
8       Gerritsen Beach
9         Highland Park
10              Madison
11    Manhattan Terrace
12          Marine Park
13          Mill Island
14        Ocean Parkway
15       Remsen Village
16                Rugby
17       Sheepshead Bay
18           Weeksville
Name: Neighborhood, dtype: object

In [ ]:
dd